In [1]:
!git clone https://github.com/huggingface/diffusers.git

Cloning into 'diffusers'...
remote: Enumerating objects: 28438, done.
remote: Counting objects: 100% (2434/2434), done.
remote: Compressing objects: 100% (1050/1050), done.
remote: Total 28438 (delta 1679), reused 1840 (delta 1208), pack-reused 26004
Receiving objects: 100% (28438/28438), 33.16 MiB | 2.41 MiB/s, done.
Resolving deltas: 100% (20554/20554), done.


In [2]:
!pip install git+https://github.com/huggingface/diffusers.git
!pip install -U -r diffusers/examples/text_to_image/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/diffusers.git to /tmp/pip-req-build-88ci84kp
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git /tmp/pip-req-build-88ci84kp
  Resolved https://github.com/huggingface/diffusers.git to commit c6ae8837512d0572639b9f57491d4482fdc8948c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.5 MB/s eta 0:00:00
  Created wheel for diffusers: filename=diffusers-0.17.0.dev0-py3-none-any.whl size=1071416 sha256=f53926adadfbe265a9741eb2d4c2fdcd53cfa4151be13178a41d0d85a9931e29
  Stored in directory: /tmp/pip-ephem-wheel-cache-qjdwz6jx/wheels/4d/b7/a8/6f9549ceec5daad78675b857ac57d697c387062506520a7b50
Successfully built diffusers
Looking in indexes: https://pypi.org/simple, 

In [9]:
!accelerate config default

2023-05-29 08:45:59.563816: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Configuration already exists at /root/.cache/huggingface/accelerate/default_config.yaml, will not override. Run `accelerate config` manually or pass a different `save_location`.


In [12]:
!export MODEL_NAME="CompVis/stable-diffusion-v1-4"
!export dataset_name="lambdalabs/pokemon-blip-captions"

!accelerate launch --mixed_precision="fp16"  diffusers/examples/text_to_image/train_text_to_image.py --pretrained_model_name_or_path="CompVis/stable-diffusion-v1-4" --dataset_name="lambdalabs/pokemon-blip-captions" --use_ema --resolution=512 --center_crop --random_flip --train_batch_size=1 --gradient_accumulation_steps=4 --gradient_checkpointing --max_train_steps=100 --learning_rate=1e-05 --max_grad_norm=1 --lr_scheduler="constant" --lr_warmup_steps=0 --output_dir="output-model"

2023-05-29 09:04:32.579674: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-29 09:04:37.910982: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:258: FutureWarning: `logging_dir` is deprecated and will be removed in version 0.18.0 of 🤗 Accelerate. Use `project_dir` instead.
  warnings.warn(
05/29/2023 09:04:41 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: fp16

{'dynamic_thresholding_ratio', 'variance_type', 'prediction_type', 'clip_sample_range', 'sample_max_value', 'thresholding'} was not found in config. Values will be initialized to default values.
{'norm_num_groups'} was not found in config. Values will be initialized to default values.
{'time_cond_proj_dim', 'encoder_hid_dim', 'time_embedding_act_fn', 'resnet_skip_time_a

In [11]:
from diffusers import StableDiffusionPipeline
import torch

model_path = "output-model"
pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16)
pipe.to("cuda")

image = pipe(prompt="metapod").images[0]
image.show()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 5>:5                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/diffusers/pipelines/pipeline_utils.py:925 in             │
│ from_pretrained                                                                                  │
│                                                                                                  │
│    922 │   │   else:                                                                             │
│    923 │   │   │   cached_folder = pretrained_model_name_or_path                                 │
│    924 │   │                                                                                     │
│ ❱  925 │   │   config_dict = cls.load_config(cached_folder)                                      │
│    926 │   │                                                                                     │
│    927 │   │   # pop out "_ignore_files" as it is only needed for download                       │
│    928 │   │   config_dict.pop("_ignore_files", None)                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/diffusers/configuration_utils.py:350 in load_config      │
│                                                                                                  │
│   347 │   │   │   ):                                                                             │
│   348 │   │   │   │   config_file = os.path.join(pretrained_model_name_or_path, subfolder, cls   │
│   349 │   │   │   else:                                                                          │
│ ❱ 350 │   │   │   │   raise EnvironmentError(                                                    │
│   351 │   │   │   │   │   f"Error no file named {cls.config_name} found in directory {pretrain   │
│   352 │   │   │   │   )                                                                          │
│   353 │   │   else:                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
OSError: Error no file named model_index.json found in directory output-model.

In [24]:
!rm -rf output-model/

In [7]:
!nvidia-smi

Mon May 29 08:35:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------